 
This notebook demonstrates how to:

1. Analyze NetCDF structure
   - Load NetCDF with geowombat
   - Examine dimensions, bands, and time series
   
2. Export time series data to raster files
   - Handle point data by creating synthetic grids
   - Export each time step as a separate GeoTIFF
   - Maintain temporal ordering and geospatial context
   
3. Extract features with xr_fresh
   - Configure comprehensive feature set for temperature data
   - Run parallel feature extraction
   - Generate feature rasters for analysis
 

In [27]:
import os
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path
import geowombat as gw
import rasterio
from rasterio.transform import from_bounds
from xr_fresh.feature_calculator_series import *
from xr_fresh.extractors_series import extract_features_series


def analyze_netcdf_structure(netcdf_path):
    """Analyze the structure of the NetCDF file to understand data layout"""
    print(f"Analyzing: {netcdf_path}")
    
    with gw.open(netcdf_path) as src:
        print(f"Dimensions: {src.dims}")
        print(f"Shape: {src.shape}")
        print(f"Available bands: {list(src.band.values)}")
        
        # Get first band to understand structure
        first_band = src.sel(band=src.band.values[0])
        print(f"\nFirst band shape: {first_band.shape}")
        print(f"First band dims: {first_band.dims}")
        
        # Check coordinates
        if hasattr(first_band, 'lat') and hasattr(first_band, 'lon'):
            print(f"Lat: {first_band.lat.values}")
            print(f"Lon: {first_band.lon.values}")
        
        # Check time dimension
        if 'time' in first_band.dims:
            print(f"Time range: {first_band.time.values[0]} to {first_band.time.values[-1]}")
            print(f"Number of time steps: {len(first_band.time)}")
            
        return src

# Analyze the NetCDF file
nc_path = '../xr_fresh/data/mean_temp.nc'
data = analyze_netcdf_structure(nc_path)

Jax is running on: gpu
Analyzing: ../xr_fresh/data/mean_temp.nc
Dimensions: ('band', 'region', 'time')
Shape: (12, 1, 151)
Available bands: ['ssp126_tg_mean_p10', 'ssp126_tg_mean_p50', 'ssp126_tg_mean_p90', 'ssp245_tg_mean_p10', 'ssp245_tg_mean_p50', 'ssp245_tg_mean_p90', 'ssp585_tg_mean_p10', 'ssp585_tg_mean_p50', 'ssp585_tg_mean_p90', 'ssp370_tg_mean_p10', 'ssp370_tg_mean_p50', 'ssp370_tg_mean_p90']

First band shape: (1, 151)
First band dims: ('region', 'time')
Lat: 61.29166793823242
Lon: -106.04166412353516
Time range: 1950-01-01T00:00:00.000000000 to 2100-01-01T00:00:00.000000000
Number of time steps: 151


In [34]:
def export_point_time_series_to_rasters(netcdf_path, band_name, output_dir, grid_size=(100, 100)):
    """
    Export time series point data to synthetic raster files for xr_fresh processing.
    
    Since this NetCDF contains point time series data (single lat/lon with time series),
    we'll create synthetic raster grids where each time step becomes a raster file
    with the point value replicated across the grid.
    
    Parameters:
    -----------
    netcdf_path : str
        Path to the NetCDF file
    band_name : str 
        Name of the band/variable to extract
    output_dir : str
        Directory to save raster files
    grid_size : tuple
        Size of synthetic grid (height, width)
    """
    
    # Create output directory
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    print(f"Exporting {band_name} from {netcdf_path}")
    print(f"Output directory: {output_dir}")
    print(f"Grid size: {grid_size}")
    
    with gw.open(netcdf_path) as src:
        # Select the specific band
        band_data = src.sel(band=band_name)
        
        # Get time series values (should be 1D array over time)
        if 'region' in band_data.dims:
            time_series = band_data.isel(region=0).values  # Take first region
        else:
            time_series = band_data.values
            
        print(f"Time series shape: {time_series.shape}")
        print(f"Time steps: {len(band_data.time)}")
        
        # Get coordinates for synthetic raster
        if hasattr(band_data, 'lat') and hasattr(band_data, 'lon'):
            center_lat = float(band_data.lat.values)
            center_lon = float(band_data.lon.values)
        else:
            center_lat, center_lon = 0.0, 0.0
        
        # Create synthetic grid bounds (small area around the point)
        buffer = 0.1  # degrees
        west = center_lon - buffer
        east = center_lon + buffer  
        south = center_lat - buffer
        north = center_lat + buffer
        
        # Create geotransform
        height, width = grid_size
        transform = from_bounds(west, south, east, north, width, height)
        
        successful_files = []
        
        # Export each time step as a raster
        for i, (time_val, data_val) in enumerate(zip(band_data.time.values, time_series)):
            try:
                # Format time for filename
                time_str = pd.to_datetime(time_val).strftime('%Y%m%d')
                filename = f"{band_name}_{time_str}.tif"
                filepath = output_path / filename
                
                # Create synthetic raster with the point value
                raster_data = np.full(grid_size, data_val, dtype=np.float32)
                
                # Write raster file
                with rasterio.open(
                    filepath,
                    'w',
                    driver='GTiff',
                    height=height,
                    width=width,
                    count=1,
                    dtype=rasterio.float32,
                    crs='EPSG:4326',
                    transform=transform,
                    compress='lzw',
                    nodata=-9999
                ) as dst:
                    dst.write(raster_data, 1)
                
                successful_files.append(str(filepath))
                
                if i % 20 == 0 or i < 5:  # Show progress
                    print(f" Exported: {filename} (value: {data_val:.2f})")
                    
            except Exception as e:
                print(f" Failed {filename}: {e}")
        
        print(f"\n Successfully exported {len(successful_files)} raster files")
        print(f"Files saved to: {output_dir}/")
        
        return successful_files

# Export the first climate scenario as rasters
band_to_export = 'ssp126_tg_mean_p10'
output_directory = 'ssp126_time_series'

exported_files = export_point_time_series_to_rasters(
    nc_path, 
    band_to_export, 
    output_directory,
    grid_size=(50, 50)  # Smaller grid for faster processing
)

Exporting ssp126_tg_mean_p10 from ../xr_fresh/data/mean_temp.nc
Output directory: ssp126_time_series
Grid size: (50, 50)
Time series shape: (151,)
Time steps: 151
 Exported: ssp126_tg_mean_p10_19500101.tif (value: -8.05)
 Exported: ssp126_tg_mean_p10_19510101.tif (value: -8.24)
 Exported: ssp126_tg_mean_p10_19520101.tif (value: -7.92)
 Exported: ssp126_tg_mean_p10_19530101.tif (value: -8.10)
 Exported: ssp126_tg_mean_p10_19540101.tif (value: -8.78)
 Exported: ssp126_tg_mean_p10_19700101.tif (value: -8.33)
 Exported: ssp126_tg_mean_p10_19900101.tif (value: -7.46)
 Exported: ssp126_tg_mean_p10_20100101.tif (value: -6.72)
 Exported: ssp126_tg_mean_p10_20300101.tif (value: -5.59)
 Exported: ssp126_tg_mean_p10_20500101.tif (value: -5.45)
 Exported: ssp126_tg_mean_p10_20700101.tif (value: -4.33)
 Exported: ssp126_tg_mean_p10_20900101.tif (value: -5.40)

 Successfully exported 151 raster files
Files saved to: ssp126_time_series/


In [45]:
# extract_features_series(
#     files=raster_files,
#     feature_list=feature_list,
#     band_name=1,  # Single band rasters
#     temp_dir=output_dir,
#     num_workers=num_workers,
#     nodata=-9999,
# )
from glob import glob
raster_files =glob("./ssp126_time_series/*.tif")
from datetime import datetime

band_name = "tg_mean"  # used to rename outputs
file_glob = f"./ssp126_time_series/*.tif"
strp_glob = f"./ssp126_time_series/ssp126_tg_mean_p10_%Y%m%d.tif"  

dates = sorted(
    datetime.strptime(string, strp_glob) for string in sorted(glob(file_glob))
)
raster_files = sorted(glob(file_glob))

print(pd.DataFrame({"date": dates, "file": raster_files}))

          date                                               file
0   1950-01-01  ./ssp126_time_series/ssp126_tg_mean_p10_195001...
1   1951-01-01  ./ssp126_time_series/ssp126_tg_mean_p10_195101...
2   1952-01-01  ./ssp126_time_series/ssp126_tg_mean_p10_195201...
3   1953-01-01  ./ssp126_time_series/ssp126_tg_mean_p10_195301...
4   1954-01-01  ./ssp126_time_series/ssp126_tg_mean_p10_195401...
..         ...                                                ...
146 2096-01-01  ./ssp126_time_series/ssp126_tg_mean_p10_209601...
147 2097-01-01  ./ssp126_time_series/ssp126_tg_mean_p10_209701...
148 2098-01-01  ./ssp126_time_series/ssp126_tg_mean_p10_209801...
149 2099-01-01  ./ssp126_time_series/ssp126_tg_mean_p10_209901...
150 2100-01-01  ./ssp126_time_series/ssp126_tg_mean_p10_210001...

[151 rows x 2 columns]


In [52]:
# Define feature set for temperature time series analysis
feature_list = {
    # Basic statistics
    "minimum": [{}],
    "maximum": [{}],
    "mean": [{}],
    "median": [{}],
    "standard_deviation": [{}],
    
    # Temporal features
    "doy_of_maximum": [{"dates": dates}],
    "doy_of_minimum": [{"dates": dates}],
    
    # Trend and change
    "mean_abs_change": [{}],
    "mean_change": [{}],
    "ols_slope_intercept": [{}],
    
    # Distribution features  
    "skewness": [{}],
    "kurtosis": [{}],
    "quantile": [{"q": 0.1}, {"q": 0.25}, {"q": 0.75}, {"q": 0.9}],
    
    # Complexity features
    "ts_complexity_cid_ce": [{}],
    "abs_energy": [{}],
    "variance_larger_than_standard_deviation": [{}],
    
    # Threshold features
    "count_above_mean": [{}],
    "count_below_mean": [{}],
    "ratio_beyond_r_sigma": [{"r": 1}, {"r": 2}]
}

print(f"Extracting {len([item for sublist in feature_list.values() for item in sublist])} features...")


# Run feature extraction
extract_features_series(
    raster_files,
    feature_list,
    band_name,
    "ssp126_time_series/features",
)

print("Feature extraction completed!")

Extracting 23 features...


RasterBlockError: The height and width of dataset blocks must be multiples of 16